In [ ]:
!pip install simpletransformers


In [ ]:
import pandas as pd
import numpy as np
import string

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')

from nltk.stem import LancasterStemmer, WordNetLemmatizer

import re


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Import Data

In [ ]:
#Creating the dataframes
kiva_train = pd.read_csv('https://raw.githubusercontent.com/DDave94/Loan-Default-Prediction/main/kiva_train.csv')
kiva_test = pd.read_csv('https://raw.githubusercontent.com/DDave94/Loan-Default-Prediction/main/kiva_test.csv')

#Formatting kaggle dataframe to run in the predict function
kaggle_test = kiva_test['en_clean']
kaggle_test = kaggle_test.to_list()

kiva_train.head(10)

,loan_id,en_clean,defaulted
0,7779,She opened a colmado out of the side of her ho...,0
1,2777,(First Loan): Joffre continues to run his loc...,1
2,6007,"Dina Santana is the mother of two children, Ju...",0
3,76,"Rosemary is 50 years old, single, and has 6 ch...",1
4,4217,"Segundo has a shop where he sells animal feed,...",0
5,5077,"I am a single parent, mother of 4. I sell groc...",0
6,6033,Mariana Jose Serda owns a general store in Mag...,0
7,843,Mary is a mother of four children. One of the ...,1
8,4357,Wilson is seeking his second loan with Kiva an...,0
9,7007,Clara lives with her two children (ages: 20 an...,0


### Preprocessing

In [ ]:
pip install unidecode textstat

In [ ]:
import unicodedata
import unidecode
stop_words = set(stopwords.words('english') + stopwords.words('spanish'))
# Lemmatization is better than stemming
lemmer = WordNetLemmatizer()
#lemmer = LancasterStemmer()

def preprocess(x):
    # Lower case
    x = x.lower()
    
    # Remove punctuation
    # x = re.sub(r'[^\w\s]', '', x)

    # Remove numbers
    # x = re.sub(r'\d+', '', x)
    
    # Remove non-unicode
    x = unidecode.unidecode(x)
        
    # Remove stopwords and lemmatize
    #x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    x = [lemmer.lemmatize(w) for w in x.split()]
    #x = [lemmer.stem(w) for w in x.split()]
    return ' '.join(x) 

kiva_train['en_clean'] = kiva_train['en_clean'].apply(preprocess)

### Looking at Training Data after preprocessing

In [ ]:
kiva_train.head(10)

,loan_id,en_clean,defaulted
0,7779,she opened a colmado out of the side of her ho...,0
1,2777,(first loan): joffre continues to run his lock...,1
2,6007,"dina santana is the mother of two children, ju...",0
3,76,"rosemary is 50 year old, single, and ha 6 chil...",1
4,4217,"segundo ha a shop where he sell animal feed, m...",0
5,5077,"i am a single parent, mother of 4. i sell groc...",0
6,6033,mariana jose serda owns a general store in mag...,0
7,843,mary is a mother of four children. one of the ...,1
8,4357,wilson is seeking his second loan with kiva an...,0
9,7007,clara life with her two child (ages: 20 and 18...,0


### Train / Val Data

In [ ]:
# Creating Training Data
X = kiva_train['en_clean']
y = kiva_train['defaulted']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.concat([X_train,y_train], axis = 1)
eval_df = pd.concat([X_val,y_val],axis = 1)

### DL Model

In [ ]:
# Create arguements
model_args = ClassificationArgs()
model_args.overwrite_output_dir = True
model_args.no_cache = True
model_args.save_model_every_epoch = False
model_args.num_train_epochs = 10
model_args.learning_rate = 4e-5
model_args.train_batch_size = 32
model_args.eval_batch_size = 32
model_args.max_seq_length = 300

model_args.manual_seed = 50

#without lemmitization (0.73)
#lemmitization with stop words (0.86)
# Lower case and pure lemmitization (0.89)

# 10 epochs (0.89)
# 12 epochs ()


# Create a Classification Model
model = ClassificationModel("xlnet", "xlnet-base-cased", args=model_args)


# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

  0%|          | 0/4910 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/154 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

Running Epoch 8 of 10:   0%|          | 0/154 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/154 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1228 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
preds, outputs = model.predict(X_val.to_list())


  0%|          | 0/1228 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_val, preds))

              precision    recall  f1-score   support

           0       0.90      0.87      0.89       629
           1       0.87      0.90      0.88       599

    accuracy                           0.89      1228
   macro avg       0.89      0.89      0.89      1228
weighted avg       0.89      0.89      0.89      1228



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_val, preds))

[[548  81]
 [ 60 539]]
